## DNN Mean + Max  
Took out data from 2020-2021, 2021-2022. (Used mean and maximum weather data)

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from statistics import mean
from statistics import mean
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from statistics import mean
import math

FUTURE_PREDICTION = 4

def findmax(x):
    maximum = 0
    for number in x:
        maximum = max(x)
    return maximum

def findmean(x):
    m = 0
    m = mean(x)
    return m

def findmin(x):
    m = 0
    m = min(x)
    return m

def CreateMean(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Mean'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            mean = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Mean'])
            mean = findmean(list)
            dataframe.at[x,f'{title}_Mean'] = mean
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMax(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Max'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            maximum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Max'])
            maximum = findmax(list)
            dataframe.at[x,f'{title}_Max'] = maximum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMin(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Min'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            minimum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Min'])
            minimum = findmin(list)
            dataframe.at[x,f'{title}_Min'] = minimum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe
    
def DataPreprocess(city):
    virus_df = pd.read_csv('Data/virus.csv')
    virus_df['Date'] = pd.to_datetime(virus_df['Date'])
    virus_df['year'] = virus_df['Date'].apply(lambda date: date.year)
    virus_df['month'] = virus_df['Date'].apply(lambda date: date.month)
    virus_df_new = virus_df[['Date','month','year',f'{city}']]
    virus_df_new.rename(columns = {f'{city}' : f'Virus_Count_{city}'}, inplace = True)

    temp_df = pd.read_csv(f'Data/{city}_temp.csv')
    pressure_df = pd.read_csv(f'Data/{city}_pressure.csv')
    hours_df = pd.read_csv(f'Data/{city}_hours.csv')
    rainfall_df = pd.read_csv(f'Data/{city}_rainfall.csv')

    temp_df['Date'] = pd.to_datetime(temp_df['Date'])
    temp_df['year'] = temp_df['Date'].apply(lambda date: date.year)
    temp_df['month'] = temp_df['Date'].apply(lambda date: date.month)

    pressure_df['Date'] = pd.to_datetime(pressure_df['Date'])
    pressure_df['year'] = pressure_df['Date'].apply(lambda date: date.year)
    pressure_df['month'] = pressure_df['Date'].apply(lambda date: date.month)
    pressure_df = pressure_df.sort_values(by="Date")
    pressure_df = pressure_df.reset_index(drop=True)

    hours_df['Date'] = pd.to_datetime(hours_df['Date'])
    hours_df['year'] = hours_df['Date'].apply(lambda date: date.year)
    hours_df['month'] = hours_df['Date'].apply(lambda date: date.month)
    hours_df = hours_df.sort_values(by="Date")
    hours_df = hours_df.reset_index(drop=True)

    rainfall_df['Date'] = pd.to_datetime(rainfall_df['Date'])
    rainfall_df['year'] = rainfall_df['Date'].apply(lambda date: date.year)
    rainfall_df['month'] = rainfall_df['Date'].apply(lambda date: date.month)
    rainfall_df = rainfall_df.sort_values(by="Date")
    rainfall_df = rainfall_df.reset_index(drop=True)
    
    list = []
    for x in temp_df.index:
        t = temp_df.iloc[x][f'{city}']
        svp = 6.1078*10**(7.5*t/(t+237.3))
        e = pressure_df.iloc[x][f'{city}']
        h = e/svp*100
        list.append(h)
        
    humidity_df = pd.DataFrame(data=list)
    humidity_df.rename(columns = {0:f'{city}'}, inplace = True)

    mean_temp_df = CreateMean(temp_df, 'Temperature',f'{city}')
    mean_pressure_df = CreateMean(pressure_df, 'Pressure',f'{city}')
    mean_hours_df = CreateMean(hours_df, 'Hours',f'{city}')
    mean_rainfall_df = CreateMean(rainfall_df, 'Rainfall',f'{city}')
    mean_humidity_df = CreateMean(humidity_df, 'Humidity',f'{city}')
    
    max_temp_df = CreateMax(temp_df, 'Temperature',f'{city}')
    max_pressure_df = CreateMax(pressure_df, 'Pressure',f'{city}')
    max_hours_df = CreateMax(hours_df, 'Hours',f'{city}')
    max_rainfall_df = CreateMax(rainfall_df, 'Rainfall',f'{city}')
    max_humidity_df = CreateMax(humidity_df, 'Humidity',f'{city}')
    

    df = pd.concat([virus_df_new,mean_humidity_df,mean_temp_df,mean_pressure_df,mean_hours_df,mean_rainfall_df,max_temp_df,max_pressure_df,max_hours_df,max_rainfall_df,max_humidity_df], axis=1)
    df = df.loc[:,~df.columns.duplicated()]

    df['VirusCRM'] = df['Virus_Count_Tokyo'].rolling(14, center = True).mean()

    for i in df.index:
        if i == 0:
            df.at[i,'Gradient'] = 1
        else:
            if df.iloc[i-1]['VirusCRM'] == 0:
                if df.at[i,'VirusCDM'] > 0:
                    df.at[i,'Gradient'] = 1
                else:
                    df.at[i,'Gradient'] = 0
            else:
                grd = df.iloc[i]['VirusCRM']/(df.iloc[i-1]['VirusCRM'])
                df.at[i,'Gradient'] = grd
    
    for i in df.index:
        if (df.iloc[i]['Gradient'] >= 1.1) and (df.iloc[i+1]['Gradient'] >= 1.1) and (df.iloc[i+2]['Gradient'] >= 1.1):
            df.at[i,'Spreading'] = 1
        else:
            df.at[i,'Spreading'] = 0
    df = df.astype({'Spreading':'int8'})

    df_virus_count = df[f'Virus_Count_{city}']
    df.drop(columns=['year','Gradient','VirusCRM',f'Virus_Count_{city}'], axis=1, inplace=True)
    df_labels = df[['Date']]
    FUTURE_PREDICTION = 4
    for i in range(FUTURE_PREDICTION):
        df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)
    temp = df[["Date", "Spreading"]]
    for i in range(FUTURE_PREDICTION):
        target_column = f"T+{(i+1)*7}"
        df_labels = df_labels.merge(temp, 
                                how ="left",
                                left_on = (target_column),
                                right_on=("Date"),
                                suffixes = ("","_y")
        )
        df_labels.rename(columns={"Spreading":f"{target_column}_Spreading"},inplace=True)
    df_labels.drop(columns=["Date_y"],inplace=True)

    for i in range(FUTURE_PREDICTION):
        df_labels.drop(columns =[f"T+{(i+1)*7}"],inplace=True,axis=1 )

    df = df.merge(df_labels,
             how='left',
             on = ["Date"])
    df.dropna(inplace=True)

    X = df.drop(columns=['T+7_Spreading','T+14_Spreading','T+21_Spreading','T+28_Spreading'],axis=1)
    y1 = df['T+21_Spreading']
    y2 = df['T+7_Spreading']
    y3 = df['T+14_Spreading']
    y4 = df['T+28_Spreading']

    return X, y1, y2, y3, y4, df, df_virus_count

In [3]:
X, y1, y2, y3, y4, df, df_virus_count = DataPreprocess('Tokyo')

/Users/miyanishio/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/k5/3yc13bs57f7637937yjlstth0000gn/T/ipykernel_86690/684746323.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)


In [4]:
X[0:400]

,Date,month,Humidity_Mean,Temperature_Mean,Pressure_Mean,Hours_Mean,Rainfall_Mean,Temperature_Max,Pressure_Max,Hours_Max,Rainfall_Max,Humidity_Max,Spreading
0,2012-09-10,9,71.007732,28.085714,26.942857,6.928571,0.928571,28.9,28.0,10.2,4.5,77.172701,0
1,2012-09-17,9,67.730487,28.414286,26.200000,8.257143,1.714286,28.8,29.2,11.4,8.5,77.708924,0
2,2012-09-24,9,78.780351,24.685714,24.600000,3.614286,19.285714,28.0,29.2,9.5,45.5,89.425667,0
3,2012-10-01,10,69.795654,23.314286,20.128571,4.928571,2.928571,26.9,24.7,11.1,13.0,79.695709,0
4,2012-10-08,10,74.175200,21.942857,19.657143,3.928571,9.214286,24.0,23.1,7.4,30.0,82.331696,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2020-04-06,4,63.967539,12.271429,9.014286,6.542857,4.428571,15.7,13.1,11.9,31.0,97.184900,0
396,2020-04-13,4,63.265454,11.557143,8.471429,6.914286,19.928571,14.3,10.5,11.7,132.0,97.882723,0
397,2020-04-20,4,68.356621,12.300000,9.728571,6.228571,15.285714,14.7,14.3,11.8,89.5,96.108231,0
398,2020-04-27,4,62.859690,13.657143,9.857143,6.671429,2.000000,18.3,13.2,12.5,12.5,81.513563,0


In [5]:
X = X.drop(columns=['Date'])

In [6]:
X = X[0:400] #until 2020 05-04
y1 = y1[0:400]
y2 = y2[0:400]
y3 = y2[0:400]
y4 = y2[0:400]

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler1 = StandardScaler()
X_train, X_test = X[0:280], X[280:400]
y1_train, y1_test = y1[0:280], y1[280:400]

In [8]:
scaled_X_train = scaler1.fit_transform(X_train)
scaled_X_test = scaler1.transform(X_test)

In [9]:
model = Sequential()

model.add(Dense(12,activation="sigmoid"))
model.add(Dense(6,activation="sigmoid"))
model.add(Dense(3,activation="sigmoid"))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

2022-11-14 17:04:33.054019: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
early_stop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=50)

In [11]:
model.fit(scaled_X_train,y1_train,epochs=400,validation_data=(scaled_X_test,y1_test),callbacks=[early_stop])

Epoch 1/400
9/9 [==============================] - 2s 55ms/step - loss: 0.5656 - accuracy: 0.8357 - val_loss: 0.5252 - val_accuracy: 0.9000
Epoch 2/400
9/9 [==============================] - 0s 8ms/step - loss: 0.5533 - accuracy: 0.8357 - val_loss: 0.5129 - val_accuracy: 0.9000
Epoch 3/400
9/9 [==============================] - 0s 10ms/step - loss: 0.5440 - accuracy: 0.8357 - val_loss: 0.5019 - val_accuracy: 0.9000
Epoch 4/400
9/9 [==============================] - 0s 10ms/step - loss: 0.5356 - accuracy: 0.8357 - val_loss: 0.4916 - val_accuracy: 0.9000
Epoch 5/400
9/9 [==============================] - 0s 10ms/step - loss: 0.5276 - accuracy: 0.8357 - val_loss: 0.4813 - val_accuracy: 0.9000
Epoch 6/400
9/9 [==============================] - 0s 8ms/step - loss: 0.5199 - accuracy: 0.8357 - val_loss: 0.4718 - val_accuracy: 0.9000
Epoch 7/400
9/9 [==============================] - 0s 9ms/step - loss: 0.5128 - accuracy: 0.8357 - val_loss: 0.4627 - val_accuracy: 0.9000
Epoch 8/400
9/9 [======

9/9 [==============================] - 0s 10ms/step - loss: 0.3890 - accuracy: 0.8357 - val_loss: 0.3164 - val_accuracy: 0.9000
Epoch 60/400
9/9 [==============================] - 0s 8ms/step - loss: 0.3875 - accuracy: 0.8357 - val_loss: 0.3155 - val_accuracy: 0.9000
Epoch 61/400
9/9 [==============================] - 0s 8ms/step - loss: 0.3860 - accuracy: 0.8357 - val_loss: 0.3144 - val_accuracy: 0.9000
Epoch 62/400
9/9 [==============================] - 0s 8ms/step - loss: 0.3846 - accuracy: 0.8357 - val_loss: 0.3135 - val_accuracy: 0.9000
Epoch 63/400
9/9 [==============================] - 0s 7ms/step - loss: 0.3831 - accuracy: 0.8357 - val_loss: 0.3127 - val_accuracy: 0.9000
Epoch 64/400
9/9 [==============================] - 0s 10ms/step - loss: 0.3815 - accuracy: 0.8357 - val_loss: 0.3115 - val_accuracy: 0.9000
Epoch 65/400
9/9 [==============================] - 0s 8ms/step - loss: 0.3800 - accuracy: 0.8357 - val_loss: 0.3104 - val_accuracy: 0.9000
Epoch 66/400
9/9 [=============

9/9 [==============================] - 0s 6ms/step - loss: 0.3077 - accuracy: 0.8357 - val_loss: 0.2675 - val_accuracy: 0.9000
Epoch 118/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3066 - accuracy: 0.8357 - val_loss: 0.2670 - val_accuracy: 0.9000
Epoch 119/400
9/9 [==============================] - 0s 7ms/step - loss: 0.3055 - accuracy: 0.8357 - val_loss: 0.2666 - val_accuracy: 0.9000
Epoch 120/400
9/9 [==============================] - 0s 9ms/step - loss: 0.3045 - accuracy: 0.8357 - val_loss: 0.2660 - val_accuracy: 0.9000
Epoch 121/400
9/9 [==============================] - 0s 10ms/step - loss: 0.3033 - accuracy: 0.8357 - val_loss: 0.2654 - val_accuracy: 0.9000
Epoch 122/400
9/9 [==============================] - 0s 8ms/step - loss: 0.3024 - accuracy: 0.8357 - val_loss: 0.2650 - val_accuracy: 0.9000
Epoch 123/400
9/9 [==============================] - 0s 9ms/step - loss: 0.3015 - accuracy: 0.8357 - val_loss: 0.2643 - val_accuracy: 0.9000
Epoch 124/400
9/9 [=======

9/9 [==============================] - 0s 7ms/step - loss: 0.2626 - accuracy: 0.8357 - val_loss: 0.2459 - val_accuracy: 0.9000
Epoch 176/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2622 - accuracy: 0.8357 - val_loss: 0.2456 - val_accuracy: 0.9000
Epoch 177/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2614 - accuracy: 0.8357 - val_loss: 0.2452 - val_accuracy: 0.9000
Epoch 178/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2610 - accuracy: 0.8357 - val_loss: 0.2451 - val_accuracy: 0.9000
Epoch 179/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2603 - accuracy: 0.8357 - val_loss: 0.2452 - val_accuracy: 0.9000
Epoch 180/400
9/9 [==============================] - 0s 9ms/step - loss: 0.2597 - accuracy: 0.8357 - val_loss: 0.2447 - val_accuracy: 0.9000
Epoch 181/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2592 - accuracy: 0.8357 - val_loss: 0.2442 - val_accuracy: 0.9000
Epoch 182/400
9/9 [=======

9/9 [==============================] - 0s 7ms/step - loss: 0.2360 - accuracy: 0.8893 - val_loss: 0.2344 - val_accuracy: 0.9083
Epoch 234/400
9/9 [==============================] - 0s 6ms/step - loss: 0.2356 - accuracy: 0.8857 - val_loss: 0.2339 - val_accuracy: 0.9167
Epoch 235/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2352 - accuracy: 0.8929 - val_loss: 0.2335 - val_accuracy: 0.9167
Epoch 236/400
9/9 [==============================] - 0s 9ms/step - loss: 0.2346 - accuracy: 0.8893 - val_loss: 0.2334 - val_accuracy: 0.9167
Epoch 237/400
9/9 [==============================] - 0s 9ms/step - loss: 0.2346 - accuracy: 0.8893 - val_loss: 0.2335 - val_accuracy: 0.9167
Epoch 238/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2341 - accuracy: 0.8893 - val_loss: 0.2339 - val_accuracy: 0.9167
Epoch 239/400
9/9 [==============================] - 0s 12ms/step - loss: 0.2335 - accuracy: 0.8929 - val_loss: 0.2338 - val_accuracy: 0.9167
Epoch 240/400
9/9 [======

9/9 [==============================] - 0s 11ms/step - loss: 0.2164 - accuracy: 0.8857 - val_loss: 0.2245 - val_accuracy: 0.9083
Epoch 292/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2161 - accuracy: 0.8893 - val_loss: 0.2246 - val_accuracy: 0.9167
Epoch 293/400
9/9 [==============================] - 0s 21ms/step - loss: 0.2157 - accuracy: 0.8857 - val_loss: 0.2245 - val_accuracy: 0.9167
Epoch 294/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2155 - accuracy: 0.8857 - val_loss: 0.2233 - val_accuracy: 0.9167
Epoch 295/400
9/9 [==============================] - 0s 9ms/step - loss: 0.2151 - accuracy: 0.8857 - val_loss: 0.2219 - val_accuracy: 0.9167
Epoch 296/400
9/9 [==============================] - 0s 13ms/step - loss: 0.2152 - accuracy: 0.8857 - val_loss: 0.2213 - val_accuracy: 0.9167
Epoch 297/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2146 - accuracy: 0.8893 - val_loss: 0.2221 - val_accuracy: 0.9167
Epoch 298/400
9/9 [====

Epoch 349/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2023 - accuracy: 0.8929 - val_loss: 0.2112 - val_accuracy: 0.9167
Epoch 350/400
9/9 [==============================] - 0s 17ms/step - loss: 0.2027 - accuracy: 0.8929 - val_loss: 0.2110 - val_accuracy: 0.9167
Epoch 351/400
9/9 [==============================] - 0s 15ms/step - loss: 0.2020 - accuracy: 0.8964 - val_loss: 0.2109 - val_accuracy: 0.9167
Epoch 352/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2017 - accuracy: 0.8929 - val_loss: 0.2106 - val_accuracy: 0.9167
Epoch 353/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2015 - accuracy: 0.8929 - val_loss: 0.2108 - val_accuracy: 0.9167
Epoch 354/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2015 - accuracy: 0.8929 - val_loss: 0.2100 - val_accuracy: 0.9167
Epoch 355/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2012 - accuracy: 0.8929 - val_loss: 0.2098 - val_accuracy: 0.9167
Epoch 356/

In [12]:
predictions = model.predict(scaled_X_test)

In [13]:
preds = []
for x in predictions:
    if x >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [14]:
from sklearn.metrics import classification_report,confusion_matrix

In [15]:
print(classification_report(y1_test,preds))

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96       108
         1.0       0.62      0.83      0.71        12

    accuracy                           0.93       120
   macro avg       0.80      0.89      0.84       120
weighted avg       0.95      0.93      0.94       120

